In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Load Data**

In [ ]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df_new = df

**Remove Null Values**

In [ ]:
#NaN values: Age, Cabin, Embarked

df['Age'].isnull().values.any()


#df_n = df.to_numpy()

In [ ]:
#df['Survived'].isnull().values.any()

#replacing age with mean
df['Age'].fillna(df['Age'].mean(), inplace=True)

In [ ]:
df

In [ ]:
#replacing age with 0 for Nan
#df['Age'].fillna(0, inplace=True)


#df_n = df.to_numpy()


#replacing Cabin with B96
counts = df['Cabin'].value_counts()
counts

df['Cabin'].fillna(0, inplace=True)

In [ ]:
# counts = df['Embarked'].value_counts()
# counts

df['Embarked'].fillna(0, inplace=True)

In [ ]:
#final check for any NaN values
df.isnull().values.any()

**Drop Non-correlated columns and add one-hot encoding (feature engineering)**

In [ ]:
#removing un-correlated features 1. PassengerId, Name, 

# df = df.drop(columns=['Name','PassengerId'])

# #y = pd.get_dummies(df.Countries, prefix='Country')
# df = df.drop(columns='Ticket')

# df = pd.get_dummies(df.Sex, prefix='Sex')

df1 = df.drop(columns=['Name','PassengerId', 'Ticket'])

In [ ]:
df1 = pd.get_dummies(df1, columns=['Sex'])

In [ ]:
df1 = pd.get_dummies(df1, columns=['Cabin', 'Embarked'])

In [ ]:
df = df1
df

**Split training and testing**

In [ ]:
#split the data into training validation

from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size=0.3)

In [ ]:
df_train_y = df_train['Survived']
df_val_y = df_val['Survived']

df_new_train_y = df_new['Survived']
df_new_train_X = df_new.drop(columns='Survived')

#df_val.drop(columns='Survived', inplace=True)

In [ ]:
df_train_X = df_train.drop(columns='Survived')
df_val_X = df_val.drop(columns='Survived')

In [ ]:
X_train = df_train_X.to_numpy()
y_train = df_train_y.to_numpy()
X_test = df_val_X.to_numpy()
y_test = df_val_y.to_numpy()

X_train_new = df_new_train_X.to_numpy()
y_train_new = df_new_train_y.to_numpy()

**Random Forests**

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=200)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

**XGBoost Classifier**

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(eta=0.2)
#model.fit(X_train, y_train)
model.fit(X_train_new, y_train_new)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#testing with actual test set

df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
df_test

In [ ]:
df_test = df_test.drop(columns=['Name','PassengerId', 'Ticket'])

#perprocessing
df_test['Age'].fillna(df['Age'].mean(), inplace=True)
df_test['Cabin'].fillna(0, inplace=True)
df_test['Embarked'].fillna(0, inplace=True)

df_test = pd.get_dummies(df_test, columns=['Sex', 'Cabin', 'Embarked'])

In [ ]:
df_test

In [ ]:
for item in df_train_X.columns:
    if item not in df_test.columns:
        df_test[item] = 0
        

for item in df_test.columns:
    if item not in df_train_X.columns:
        df_test = df_test.drop(columns=item)

        
        
#df_test = df_test.reindex(columns = ['Cabin_A10'])   
#header_list
#df_test

In [ ]:
len(df_test.columns)

In [ ]:
len(df_train_X.columns)

In [ ]:
df_test

In [ ]:
X_test = df_test.to_numpy()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
df_test_final = pd.DataFrame(y_pred)

In [ ]:
df_test_final.index = df_test_final.index + 892

In [ ]:
df_test_final = df_test_final.reset_index()

In [ ]:
df_test_final.columns = ['PassengerId', 'Survived']

In [ ]:
df_test_final

In [ ]:
df_test_final.to_csv('results.csv')